# Получение низкоразмерных фич из изображений персон после процессинга видео

In [1]:
import os, sys, inspect
sys.path.insert(0, '..')
from generate_detections import *
import numpy as np
import cv2
sys.path.append('C:\\my_cv_work\\tpe_recognition\\bin\\tpe')
from tpe_train import *

Using TensorFlow backend.


## Предобработка файлов

In [2]:
im_root_path = 'D:/data/pics/persons_yolo_test' # директория с папками, номер папки - индекс класса

In [3]:
class_data = {folder: len(os.listdir(os.path.join(im_root_path, folder))) for folder in os.listdir(im_root_path)}
# класс - кол-во экземпляров
MIN_COUNT = 10 # минимальное кол-во изображений в классе

## Создание фич

In [4]:
model_path="../resources/networks/mars-small128.ckpt-68577"
encoder = create_box_encoder(model_path, batch_size=32, loss_mode='cosine') # нейросеть для создания фич

feature dimensionality:  128


In [5]:
# <bb_left>, <bb_top>, <bb_width>, <bb_height>,

In [6]:
saving_path = ''

In [38]:
img_embeddings = None
for k,v in class_data.items():
    print('processing class {}'.format(k))
    if v >= MIN_COUNT:
        class_features = None
        labels = []
        for img in os.listdir(os.path.join(im_root_path, k)):
            img_ary = cv2.imread(os.path.join(im_root_path, k,img))
            if img_ary is None:
                continue
            labels.append(k)
            bbox = np.array([0,0,img_ary.shape[1], img_ary.shape[0]]).reshape(1,4)
            features = encoder(img_ary, bbox)
            if class_features is None:
                class_features = features
            else:
                class_features = np.vstack([class_features, features])
        np_labels = np.asarray(labels).reshape(len(labels), 1)
        class_features = np.hstack([np_labels, class_features])
        if img_embeddings is None:
            img_embeddings = class_features
        else:
            img_embeddings = np.vstack([img_embeddings, class_features])


processing class 1
processing class 10
processing class 1000
processing class 1001
processing class 1002
processing class 1003
processing class 1005
processing class 1006
processing class 1008
processing class 1009
processing class 1010
processing class 1011
processing class 1012
processing class 1013
processing class 1016
processing class 1017
processing class 1020
processing class 1021
processing class 1022
processing class 1023
processing class 1024
processing class 1025
processing class 1028
processing class 1029
processing class 1030
processing class 1031
processing class 1032
processing class 1033
processing class 1035
processing class 1036
processing class 1038
processing class 1039
processing class 1040
processing class 1042
processing class 1043
processing class 1045
processing class 1046
processing class 1047
processing class 1048
processing class 1049
processing class 1050
processing class 1051
processing class 1054
processing class 1055
processing class 1056
processing clas

processing class 1479
processing class 148
processing class 1480
processing class 1482
processing class 1484
processing class 1485
processing class 1486
processing class 1488
processing class 1489
processing class 149
processing class 1491
processing class 1492
processing class 1494
processing class 1495
processing class 1497
processing class 15
processing class 150
processing class 1501
processing class 1502
processing class 1503
processing class 1504
processing class 1508
processing class 151
processing class 1511
processing class 1512
processing class 1513
processing class 1514
processing class 1515
processing class 1516
processing class 1518
processing class 1519
processing class 1521
processing class 1524
processing class 1525
processing class 1526
processing class 1528
processing class 1531
processing class 1532
processing class 1533
processing class 1534
processing class 1536
processing class 1539
processing class 154
processing class 1540
processing class 1541
processing class 

processing class 1988
processing class 1989
processing class 199
processing class 1992
processing class 1993
processing class 1994
processing class 1995
processing class 1996
processing class 1998
processing class 1999
processing class 2
processing class 200
processing class 2000
processing class 2001
processing class 2002
processing class 2003
processing class 2005
processing class 2006
processing class 2007
processing class 2008
processing class 2009
processing class 201
processing class 2010
processing class 2011
processing class 2012
processing class 2014
processing class 2015
processing class 2018
processing class 2021
processing class 2022
processing class 2023
processing class 2024
processing class 2026
processing class 2027
processing class 2028
processing class 2029
processing class 203
processing class 2030
processing class 2032
processing class 2033
processing class 2034
processing class 2038
processing class 2040
processing class 2041
processing class 2043
processing class 

processing class 647
processing class 649
processing class 65
processing class 650
processing class 651
processing class 652
processing class 653
processing class 655
processing class 656
processing class 658
processing class 66
processing class 660
processing class 661
processing class 662
processing class 663
processing class 664
processing class 666
processing class 667
processing class 670
processing class 671
processing class 673
processing class 674
processing class 675
processing class 676
processing class 677
processing class 678
processing class 679
processing class 68
processing class 680
processing class 681
processing class 682
processing class 683
processing class 684
processing class 685
processing class 686
processing class 688
processing class 691
processing class 692
processing class 693
processing class 695
processing class 697
processing class 698
processing class 699
processing class 7
processing class 70
processing class 703
processing class 704
processing class 70

In [41]:
np.save(os.path.join(saving_path, 'embeddings.npy'), img_embeddings)

## Тест распаковки фич

In [7]:
data = np.load(os.path.join(saving_path, 'embeddings.npy'))

In [8]:
labels = data[:,0].astype('int')
embs = data[:,1:].astype('float')

In [9]:
labels, embs, nclasses, nfeatures, nexamples = unpack_features(os.path.join(saving_path, 'embeddings.npy'))

In [10]:
train_labels, train_embs, dev_labels, dev_embs = train_dev_split(embs, labels)

In [11]:
dev_labels.shape[0]

5673

In [12]:
assert train_labels.shape[0] == train_embs.shape[0]
assert dev_labels.shape[0] == dev_embs.shape[0]

In [13]:
protocol = make_protocol(dev_labels)

In [14]:
from tpe_model_builder import *

In [15]:
tpe_in_shape = 128
tpe_out_shape = 128
pca = PCA(tpe_out_shape)
pca.fit(train_embs)
W_pca = pca.components_
tpe_model, tpe_predictor = build_tpe(tpe_in_shape, tpe_out_shape, W_pca)

C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=128, weights=[array([[ ..., activation="linear", use_bias=False)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
C:\ProgramData\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf

In [16]:
nepoch=10
batch_size = 32
sampling_batch_size = 100
model_saving_path = 'D:\\data\\models\\tpe_classifier'
model_saving_name = 'run_10_epochs.h5'

In [17]:
mineer, mindeer = train_tpe(
    tpe_model, tpe_predictor, train_embs, dev_embs, protocol, train_labels,
    nclasses, nepoch, batch_size, sampling_batch_size, model_saving_path, model_saving_name)

epoch: 0
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7785
min score: -0.260694295167923, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7057
min score: -0.2387014478445053, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6961
min score: -0.23644007742404938, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7483
min score: -0.19242191314697266, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7062
min score: -0.19648832082748413, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7028
min score: -0.19893145561218262, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6716
min score: -0.2229088544845581, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7172
mi

32/32 [==============================] - 0s - loss: 0.6795
min score: -0.4720343351364136, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7153
min score: -0.46789368987083435, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7084
min score: -0.46186894178390503, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7365
min score: -0.44649073481559753, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6639
min score: -0.46270373463630676, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6914
min score: -0.4741503894329071, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6799
min score: -0.47465306520462036, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7020
min score: -0.47762

32/32 [==============================] - 0s - loss: 0.6880
min score: -0.5340214967727661, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6859
min score: -0.5318419933319092, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7144
min score: -0.5367615818977356, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6891
min score: -0.5386106967926025, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6928
min score: -0.5377505421638489, max score: 1.0000008344650269
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6815
min score: -0.543548047542572, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7142
min score: -0.5330114364624023, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6847
min score: -0.53097563982

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7103
min score: -0.5276556611061096, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7133
min score: -0.5213955044746399, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6843
min score: -0.5223490595817566, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6889
min score: -0.521803617477417, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6876
min score: -0.5213433504104614, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7032
min score: -0.517559289932251, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6999
min score: -0.5140054821968079, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7035
min score: -0.52

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6908
min score: -0.5605496168136597, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7230
min score: -0.5577474236488342, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6846
min score: -0.552009642124176, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7051
min score: -0.5584337711334229, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6908
min score: -0.5528308749198914, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6461
min score: -0.5647178888320923, max score: 1.0000008344650269
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6480
min score: -0.5828983187675476, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7089
min score: -0.5

...

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6943
min score: -0.5653724670410156, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7225
min score: -0.5504022240638733, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6824
min score: -0.5688813924789429, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6475
min score: -0.5710027813911438, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6671
min score: -0.5649971961975098, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6755
min score: -0.5715057849884033, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6979
min score: -0.567221999168396, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6802
min score: -0.5

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6612
min score: -0.5991249680519104, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6795
min score: -0.5875487327575684, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7165
min score: -0.5757982730865479, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6926
min score: -0.5683537125587463, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6818
min score: -0.5604195594787598, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6753
min score: -0.5622337460517883, max score: 1.0000008344650269
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7073
min score: -0.5724002122879028, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6692
min score: -0.

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6930
min score: -0.6096082329750061, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6754
min score: -0.6075125932693481, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6885
min score: -0.6008594632148743, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7156
min score: -0.59629887342453, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6709
min score: -0.6044077277183533, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6881
min score: -0.5953377485275269, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6706
min score: -0.6041850447654724, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6715
min score: -0.59

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6975
min score: -0.573590099811554, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6789
min score: -0.5803054571151733, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6951
min score: -0.5776406526565552, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6641
min score: -0.5967056155204773, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7413
min score: -0.5685800313949585, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6792
min score: -0.5706719160079956, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6712
min score: -0.5587736368179321, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6730
min score: -0.5

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6559
min score: -0.5695542693138123, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6937
min score: -0.5650440454483032, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7033
min score: -0.5666382908821106, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6749
min score: -0.5728943347930908, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7063
min score: -0.5650721788406372, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6545
min score: -0.5820476412773132, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6745
min score: -0.5813968777656555, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6699
min score: -0.

In [18]:
mineer

0.12183380669577182

In [19]:
mindeer

0.30296179983350968

In [20]:
_, tpe_predictor_trained = build_tpe(tpe_in_shape, tpe_out_shape)

C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=128, weights=[array([[ ..., activation="linear", use_bias=False)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
C:\ProgramData\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\my_cv_work\tpe_recognition\bin\tpe\tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf

In [22]:
tpe_predictor_trained.load_weights(os.path.join(model_saving_path, model_saving_name))

In [23]:
test_dir = 'D:\\data\\pics\\persons_yolo_test\\2040'

In [27]:
test_embs = None
for im_file in os.listdir(test_dir):
    img = cv2.imread(os.path.join(test_dir,im_file))
    bbox = np.array([0,0,img.shape[1], img.shape[0]]).reshape(1,4)
    features = encoder(img, bbox)
    if test_embs is None:
        test_embs = features
    else:
        test_embs = np.vstack([test_embs, features])

In [29]:
from itertools import combinations

In [31]:
test_embs.shape

(50, 128)

In [33]:
cc = 0
wrongs = 0
for a,b in combinations(test_embs, 2):
    a = a.reshape(1,128)
    b = b.reshape(1,128)
    a_tpe = tpe_predictor_trained.predict(a)
    b_tpe = tpe_predictor_trained.predict(b)
    scores = a_tpe @ b_tpe.T
    cc +=1
    if scores<mindeer:
        wrongs += 1
print(wrongs/cc)

0.0032653061224489797


In [36]:
test_dir_2 = 'D:\\data\\pics\\persons_yolo_test\\2034'

In [37]:
test_embs_2 = None
for im_file in os.listdir(test_dir_2):
    img = cv2.imread(os.path.join(test_dir_2,im_file))
    bbox = np.array([0,0,img.shape[1], img.shape[0]]).reshape(1,4)
    features = encoder(img, bbox)
    if test_embs is None:
        test_embs_2 = features
    else:
        test_embs_2 = np.vstack([test_embs, features])

In [38]:
xx, yy = np.meshgrid(test_embs, test_embs_2)

In [47]:
z = [
    int((tpe_predictor_trained.predict(a.reshape(1,128)) @ tpe_predictor_trained.predict(b.reshape(1,128)).T < mindeer)[0][0]) for a in test_embs for b in test_embs_2
]

In [49]:
len(z)

2550

In [51]:
sum(z)/len(z)

0.017254901960784313

In [62]:
sys.path.append('C:\\my_cv_work\\tpe_recognition\\bin\\tpe')
sys.path.append('C:\\my_cv_work\\tpe_recognition\\bin\\deep_sort')

In [69]:
from person_verificator import person_verificator

ImportError: attempted relative import with no known parent package

In [75]:
sys.path.append('C:\\my_cv_work\\tpe_recognition\\pkg_a')

In [79]:
from pkg_a import sub_a

ModuleNotFoundError: No module named 'tpe_recognition'